<a href="https://colab.research.google.com/github/tcheung99/ResNet_MiniProject/blob/master/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
%load_ext tensorboard

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf 
import numpy as np 
print(tf.__version__)

1.15.2


In [0]:
import tarfile
from six.moves import urllib
import sys
import numpy as np
import _pickle as cPickle
import os
import cv2

In [0]:
from datetime import datetime
import time
import pandas as pd

**Formulation** 


---


Based on He et. al's *Deep Residual Learning for Image Recognition*.

General:

*   Full pre-activation residual block
*   No bottleneck residual blocks
*   Perform 1x1 convolution insteaad of padding input (projection shortcut) for matching output size 


*   1st layer is 3x3 convolutions 
*   Stack of 6n layers with 3x3 on feature maps of size {32,16,8} respectively (n = number of residual blocks) 


*   Ends with global average pooling, a FC layer, and softmax
*   List item





CIFAR-10:

*   Image size = 32x32
*   Label Dimension = 10





# Loading training and test data for the CIFAR10 dataset

In [5]:
data_dir = 'cifar10_data'
full_data_dir = data_dir + '/cifar-10-batches-py/data_batch_'
valid_dir = data_dir + '/cifar-10-batches-py/test_batch'
URL = 'http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'

img_width = 32
img_height = 32
img_depth = 3

num_train_batches = 5 
num_epochs = 10000 * num_train_batches
num_classes = 10

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
filename = 'cifar-10-python.tar.gz'
filepath = os.path.join(data_dir, filename)

if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
        sys.stdout.write('\r>> Downloading %s %.1f%%' % (filename, float(count * block_size)/ float(total_size) * 100.0))
        sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
    tarfile.open(filepath, 'r:gz').extractall(data_dir)

def read_one_batch(path, use_random_label):
    '''
    The training data contains five data batches in total. The validation data has only one
    batch. This function takes the directory of one batch of data and returns the images and
    corresponding labels as numpy arrays
    :param path: the directory of one batch of data
    :param is_random_label: do you want to use random labels?
    :return: image numpy arrays and label numpy arrays
    '''
    infile = open(path, 'rb')
    new_dict = cPickle.load(infile, encoding='latin1')
    infile.close()

    image = new_dict['data']
    if use_random_label: # Add random labels 
        labels = np.random.randint(low=0, high=10, size=10000)
        label = np.array(labels)
    else:
        label = np.array(new_dict['labels'])
    
    return image, label


def read_all(all_paths, shuffle=True, use_random_label = False):
    """
    This function reads all training or validation data, shuffles them if needed, and returns the
    images and the corresponding labels as numpy arrays
    :param address_list: a list of paths of cPickle files
    :return: concatenated numpy array of data and labels. Data are in 4D arrays: [num_images,
    image_height, image_width, image_depth] and labels are in 1D arrays: [num_images]
    """
    data = np.array([]).reshape([0, img_width * img_height * img_depth])
    label = np.array([])

    for path in all_paths:
        batch_data, batch_label = read_one_batch(path, use_random_label)
        # np.concatenate concatenates along the 0-axis
        data = np.concatenate((data, batch_data))
        label = np.concatenate((label, batch_label))

    num_data = len(label)

    data = data.reshape((num_data, img_height * img_width, img_depth), order='F')
    data = data.reshape((num_data, img_height, img_width, img_depth))

    if shuffle:
        print('Shuffling')
        order = np.random.permutation(num_data)
        data = data[order, ...]
        label = label[order]

    data = data.astype(np.float32)
    return data, label

def random_crop_flip(batch_images, padding_size):
    '''
    Helper to random crop and random flip a batch of images
    :param padding_size: int. how many layers of 0 padding was added to each side
    :param batch_data: a 4D batch array
    :return: randomly cropped and flipped image
    '''
    cropped_batch = np.zeros(len(batch_images) * img_height * img_width * img_depth).reshape(len(batch_images), img_height, img_width, img_depth)
    flip_prop = np.random.randint(low=0, high=2)

    for i in range(len(batch_images)):
        x_offset = np.random.randint(low=0, high=2 * padding_size, size=1)[0]
        y_offset = np.random.randint(low=0, high=2 * padding_size, size=1)[0]
        cropped_batch[i, ...] = batch_images[i, ...][x_offset:x_offset+img_height,y_offset:y_offset+img_width, :]

    if flip_prop == 0:
        cropped_batch[i, ...] = cv2.flip(cropped_batch[i, ...], 1)

    return cropped_batch

>> Downloading cifar-10-python.tar.gz 100.0%
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.


Define Training Hyperparameters 

In [0]:
FLAGS = tf.app.flags.FLAGS

In [0]:
# DEFINE args: flagname, defaultvalue, docstring
# tf.app.flags.DEFINE_integer('max_train_steps', 70000, 'Number of steps for training')

# tf.app.flags.DEFINE_float('lr_init', 0.1, 'Initial learning rate')
# tf.app.flags.DEFINE_float('lr_decay_fact', 0.1, 'Factor to decay the learning rate by')

# tf.app.flags.DEFINE_integer('num_residual_block', 2, 'Number of residual blocks')
# tf.app.flags.DEFINE_float('weight_decay', 0.00025, 'l2 regularization')


## The following flags are related to save paths, tensorboard outputs and screen outputs

tf.app.flags.DEFINE_string('version', 'test_3', '''A version number defining the directory to save
logs and checkpoints''')
tf.app.flags.DEFINE_integer('report_freq', 400, '''Steps takes to output errors on the screen
and write summaries''')
tf.app.flags.DEFINE_float('train_ema_decay', 0.95, '''The decay factor of the train error's
moving average shown on tensorboard''')


## The following flags define hyper-parameters regards training

tf.app.flags.DEFINE_integer('train_steps', 20000, 'Number of steps for training')
tf.app.flags.DEFINE_boolean('is_full_validation', False, '''Validation w/ full validation set or a random batch''')
tf.app.flags.DEFINE_integer('train_batch_size', 128, 'Train batch size')
tf.app.flags.DEFINE_integer('validation_batch_size', 250, 'Validation batch size')
tf.app.flags.DEFINE_integer('test_batch_size', 125, 'Test batch size')

tf.app.flags.DEFINE_float('init_lr', 0.1, 'Initial learning rate')
tf.app.flags.DEFINE_float('lr_decay_factor', 0.1, 'Factor to decay the learning rate')
tf.app.flags.DEFINE_integer('decay_step0', 11000, 'First step to decay the learning rate')
tf.app.flags.DEFINE_integer('decay_step1', 16000, 'Second step to decay the learning rate')


## The following flags define hyper-parameters modifying the training network

tf.app.flags.DEFINE_integer('num_residual_blocks', 2, 'Number of residual blocks')
# tf.app.flags.DEFINE_float('weight_decay', 0.0002, '''scale for l2 regularization''')
tf.app.flags.DEFINE_float('weight_decay', 0.0001, 'Scale for l2 regularization')


## The following flags are related to data-augmentation

tf.app.flags.DEFINE_integer('padding_size', 2, '''In data augmentation, layers of zero padding on each side of the image''')


## If you want to load a checkpoint and continue training

tf.app.flags.DEFINE_string('ckpt_path', 'cache/logs_repeat20/model.ckpt-100000', '''Checkpoint
directory to restore''')
tf.app.flags.DEFINE_boolean('is_use_ckpt', False, '''Whether to load a checkpoint and continue
training''')

tf.app.flags.DEFINE_string('test_ckpt_path', 'model_110.ckpt-79999', '''Checkpoint
directory to restore''')

tf.app.flags.DEFINE_string('f', '', 'kernel')

train_dir = 'logs_' + FLAGS.version + '/'

# Define ResNet functions

A residual block is when activation of a layer is "fast-forwarded" to a deeper layer 

*   Abstract to passing input through BN, relu, conv 



In [8]:
def make_var(name,shape,initializer=tf.contrib.layers.xavier_initializer(),is_fc_layer=False):
  # initializer = tf.contrib.layers.xavier_initializer()
  regularizer = tf.contrib.layers.l2_regularizer(scale=FLAGS.weight_decay)
  new_var = tf.get_variable(name, shape=shape, initializer=initializer, regularizer=regularizer)
  return new_var

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
def bn_layer(input_layer, input_depth):
  # Batch norm layer 
  mean, var = tf.nn.moments(input_layer, axes=[0,1,2])
  beta = tf.get_variable('beta', input_depth, tf.float32, tf.constant_initializer(0.0,tf.float32))
  gamma = tf.get_variable('gamma', input_depth, tf.float32, tf.constant_initializer(1.0,tf.float32))
  BN_layer = tf.nn.batch_norm_with_global_normalization(input_layer, mean, var, beta,gamma, 0.001)
  return BN_layer

In [0]:
def conv_layer(input_layer, input_depth, filter_shape,stride):
  #conv(relu(bn(input_layer)))

  BN_layer = bn_layer(input_layer, input_depth)

  # Apply ReLu 
  ReLu_layer = tf.nn.relu(BN_layer)

  # Conv layer 
  filter = make_var(name='conv',shape=filter_shape)
  conv_layer = tf.nn.conv2d(ReLu_layer, filter=filter,strides=[1,stride,stride,1],padding='SAME')

  return conv_layer

In [0]:
def res_block(input_layer, output_depth, first_block=False):
  input_depth = input_layer.get_shape().as_list()[-1]
  if input_depth == output_depth:
    project = False 
    stride = 1 
  elif input_depth != output_depth:
    project = True 
    stride = 2 
  with tf.variable_scope('conv1_in'):
    if first_block:
      filter = make_var('conv',[3,3,input_depth,output_depth])    
      conv1 = tf.nn.conv2d(input_layer,filter=filter,strides=[1,1,1,1],padding='SAME')
    elif not first_block:
      conv1 = conv_layer(input_layer,input_depth, [3,3,input_depth,output_depth],stride)

  with tf.variable_scope('conv2_in'):
    conv2 = conv_layer(conv1, conv1.get_shape().as_list()[-1], [3,3,output_depth,output_depth],1)

  if project == True:
    # 1x1 conv shortcut projection 
    # mod_input = tf.nn.conv2d(input_layer, [1,1,input_depth,output_depth], stride)

    mod_input = conv_layer(input_layer, input_depth, [1,1,input_depth,output_depth],2)

    # pooled_input = tf.nn.avg_pool(input_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    # padded_input = tf.pad(pooled_input, [[0, 0], [0, 0], [0, 0], [input_depth // 2,input_depth // 2]])                                                          
    # mod_input = padded_input
  elif project == False:
    mod_input = input_layer

  result = conv2 + mod_input
  return result

In [0]:
def fc_layer(input_layer, dimensions):
  input_depth = input_layer.get_shape().as_list()[-1]
  BN_layer = bn_layer(input_layer, input_depth)
  ReLu_layer = tf.nn.relu(BN_layer)
  pool_layer = tf.reduce_mean(ReLu_layer, [1, 2])

  assert pool_layer.get_shape().as_list()[-1:] == [64]
  ##
  fc_w = make_var(name='fc_weights', shape=[input_depth, dimensions],initializer=tf.uniform_unit_scaling_initializer(factor=1.0), is_fc_layer=True)
  fc_b = make_var(name='fc_bias', shape=[dimensions], initializer=tf.zeros_initializer())
  fc_h = tf.nn.softmax(tf.matmul(pool_layer, fc_w) + fc_b)
  
  return fc_h

In [0]:
def tensor_summary(t):
  name = t.op.name
  tf.summary.histogram(name+'/activations', t)
  tf.summary.scalar(name + '/sparsity', tf.nn.zero_fraction(t))

In [0]:
def resnet(input_tensor,n,reuse):

  all_layers = list()

  # Op on input tensor
  with tf.variable_scope('conv0',reuse=reuse):
    filter = make_var(name='conv',shape=[3,3,3,16])
    conv_layer = tf.nn.conv2d(input_tensor, filter, strides=[1, 1, 1, 1], padding='SAME')
    BN_layer = bn_layer(conv_layer, 16)
    conv0 = tf.nn.relu(BN_layer)
    tensor_summary(conv0)
    all_layers.append(conv0)

  for i in range(n):
    with tf.variable_scope('conv1_%d' %i, reuse=reuse):
      if i == 0:
          conv1 = res_block(all_layers[-1], 16, first_block=True)
      else:
          conv1 = res_block(all_layers[-1], 16)
      tensor_summary(conv1)
      all_layers.append(conv1)

  for i in range(n):
    with tf.variable_scope('conv2_%d' %i, reuse=reuse):
      conv2 = res_block(all_layers[-1], 32)
      tensor_summary(conv2)
      all_layers.append(conv2)

  for i in range(n):
    with tf.variable_scope('conv3_%d' %i, reuse=reuse):
      conv3 = res_block(all_layers[-1], 64)
      tensor_summary(conv3)
      all_layers.append(conv3)
    assert conv3.get_shape().as_list()[1:] == [8, 8, 64]

  with tf.variable_scope('fc', reuse=reuse):
    fc = fc_layer(all_layers[-1],10)
    all_layers.append(fc)

  return all_layers[-1]

In [0]:
def test_graph(train_dir='logs'):
  input_tensor = tf.constant(np.ones([128, 32, 32, 3]), dtype=tf.float32)
  output = resnet(input_tensor, reuse=False, n=2)
  init = tf.initialize_all_variables()
  session = tf.Session()
  session.run(init)
  summary_writer = tf.train.SummaryWriter(train_dir, sess.graph)

# Training and Validation 


In [0]:
class Train_ResNet(object):
  def __init__(self):
    # Initialize placeholders
    self.lr_ph = tf.placeholder(dtype=tf.float32, shape=[])
    self.image_ph = tf.placeholder(dtype=tf.float32, shape=[FLAGS.train_batch_size, img_height,img_width, img_depth])
    self.label_ph = tf.placeholder(dtype=tf.int32, shape=[FLAGS.train_batch_size])
    self.valid_image_ph = tf.placeholder(dtype=tf.float32, shape=[FLAGS.validation_batch_size, img_height, img_width, img_depth])
    self.valid_label_ph = tf.placeholder(dtype=tf.int32, shape=[FLAGS.validation_batch_size])

  def build_train_valid_graph(self):
    global_step = tf.Variable(0, trainable=False)
    validation_step = tf.Variable(0, trainable=False)

    logits = resnet(self.image_ph, FLAGS.num_residual_blocks, reuse=False)
    
    # Reuse weight from training, reuse=True 
    valid_logits = resnet(self.valid_image_ph, FLAGS.num_residual_blocks, reuse=True)

    # Training loss 
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = self.loss(logits, self.label_ph)
    self.all_loss = tf.add_n([loss] + reg_losses)

    predictions = tf.nn.softmax(logits)
    self.train_top1_error, self.train_acc = self.top_k_error(predictions, self.label_ph, 1)

    # Validation loss
    self.valid_loss = self.loss(valid_logits, self.valid_label_ph)
    vali_predictions = tf.nn.softmax(valid_logits)
    self.vali_top1_error, self.val_acc = self.top_k_error(vali_predictions, self.valid_label_ph, 1)

    self.train_op, self.train_ema_op = self.train_operation(global_step, self.all_loss,self.train_top1_error,self.train_acc)
    self.val_op = self.validation_operation(validation_step, self.vali_top1_error, self.valid_loss,self.val_acc)


  def train(self):
    train_paths = list() 
    for i in range(1, num_train_batches+1):
        train_paths.append(full_data_dir + str(i))
    all_data, all_labels = read_all(train_paths, use_random_label=False)     
    
    pad_width = ((0, 0), (FLAGS.padding_size, FLAGS.padding_size), (FLAGS.padding_size, FLAGS.padding_size), (0, 0))
    all_data = np.pad(all_data, pad_width=pad_width, mode='constant', constant_values=0)

    valid_data, valid_labels = read_all([valid_dir],use_random_label=False)

    # Build train and validation graph
    self.build_train_valid_graph()

    # Saver to save checkpoints 
    saver = tf.train.Saver(tf.global_variables())
    summary_op = tf.summary.merge_all()
    init = tf.initialize_all_variables()
    sess = tf.Session()

    # Loading from a checkpoint
    if FLAGS.is_use_ckpt is True:
      saver.restore(sess, FLAGS.ckpt_path)
      print ('Restored from checkpoint')
    else:
      sess.run(init)

    summary_writer = tf.summary.FileWriter(train_dir, sess.graph)

    step_list = []
    train_error_list = []
    val_error_list = []

    print ('Training')
    print ('-----------------------------------')

    for step in range(FLAGS.train_steps):
      train_offset = np.random.choice(num_epochs - FLAGS.train_batch_size, 1)[0]
      batch_data = all_data[train_offset:train_offset + FLAGS.train_batch_size, ...]
      train_batch_data = random_crop_flip(batch_data, padding_size=FLAGS.padding_size)
      train_batch_labels = all_labels[train_offset:train_offset+FLAGS.train_batch_size]

      val_offset = np.random.choice(10000 - FLAGS.validation_batch_size, 1)[0]
      validation_batch_data = valid_data[val_offset:val_offset+FLAGS.validation_batch_size, ...]
      validation_batch_labels = valid_labels[val_offset:val_offset+FLAGS.validation_batch_size]

      # Want to validate once before training. You may check the theoretical validation
      # loss first
      if step % FLAGS.report_freq == 0:
        if FLAGS.is_full_validation is True:
          validation_loss_value, validation_error_value = self.full_validation(
                                        loss=self.valid_loss,
                                        top1_error=self.vali_top1_error, vali_data=valid_data,
                                        vali_labels=valid_labels, session=sess,
                                        batch_data=train_batch_data, batch_label=train_batch_labels)
          vali_summ = tf.Summary()
          vali_summ.value.add(tag='full_validation_error',
                              simple_value=validation_error_value.astype(np.float))
          summary_writer.add_summary(vali_summ, step)
          summary_writer.flush()

        else:
          _, validation_error_value, validation_loss_value = sess.run(
                                        [self.val_op,self.vali_top1_error,self.valid_loss],
                                        {self.image_ph: train_batch_data,
                                          self.label_ph: train_batch_labels,
                                          self.valid_image_ph: validation_batch_data,
                                          self.valid_label_ph: validation_batch_labels,
                                          self.lr_ph: FLAGS.init_lr})

          val_error_list.append(validation_error_value)

      start_time = time.time()
      _, _, train_loss_value, train_error_value = sess.run(
                                          [self.train_op, self.train_ema_op,self.all_loss, self.train_top1_error],
                                          {self.image_ph: train_batch_data,
                                          self.label_ph: train_batch_labels,
                                          self.valid_image_ph: validation_batch_data,
                                          self.valid_label_ph: validation_batch_labels,
                                          self.lr_ph: FLAGS.init_lr})
      duration = time.time() - start_time

      if step % FLAGS.report_freq == 0:
        summary_str = sess.run(summary_op, {self.image_ph: train_batch_data,
                                            self.label_ph: train_batch_labels,
                                            self.valid_image_ph: validation_batch_data,
                                            self.valid_label_ph: validation_batch_labels,
                                            self.lr_ph: FLAGS.init_lr})
        summary_writer.add_summary(summary_str, step)

        num_examples_per_step = FLAGS.train_batch_size
        examples_per_sec = num_examples_per_step / duration
        sec_per_batch = float(duration)

        format_str = ('%s: step %d, loss = %.4f (%.1f examples/sec; %.3f ' 'sec/batch)')
        print (format_str % (datetime.now(), step, train_loss_value, examples_per_sec,sec_per_batch))
        print ('Train top1 err = ', train_error_value)
        print ('Valid top1 err = %.4f' % validation_error_value)
        print ('Valid loss = ', validation_loss_value)
        print ('-----------------------------------')

        step_list.append(step)
        train_error_list.append(train_error_value)

      if step == FLAGS.decay_step0 or step == FLAGS.decay_step1:
        FLAGS.init_lr = 0.1 * FLAGS.init_lr
        print ('Learning rate after decay ', FLAGS.init_lr)

      # Save checkpoints every 10000 steps
      if step % 10000 == 0 or (step + 1) == FLAGS.train_steps:
        checkpoint_path = os.path.join(train_dir, 'model.ckpt')
        saver.save(sess, checkpoint_path, global_step=step)

        df = pd.DataFrame(data={'step':step_list, 'train_error':train_error_list,'validation_error': val_error_list})
        df.to_csv(train_dir + FLAGS.version + '_error.csv')


  def test(self, test_image_array):
    num_test_images = len(test_image_array)
    num_batches = num_test_images // FLAGS.test_batch_size
    remain_images = num_test_images % FLAGS.test_batch_size
    print ('%i test batches in total...' %num_batches)

    # Create the test image and labels placeholders
    self.test_image_placeholder = tf.placeholder(dtype=tf.float32, shape=[FLAGS.test_batch_size, img_height, img_width, img_depth])

    # Build the test graph
    logits = resnet(self.test_image_placeholder, FLAGS.num_residual_blocks, reuse=False)
    predictions = tf.nn.softmax(logits)

    # Initialize a new session and restore a checkpoint
    saver = tf.train.Saver(tf.all_variables())
    sess = tf.Session()

    saver.restore(sess, FLAGS.test_ckpt_path)
    print ('Model restored from ', FLAGS.test_ckpt_path)

    prediction_array = np.array([]).reshape(-1, num_classes)
    # Test by batches
    for step in range(num_batches):
      if step % 10 == 0:
        print ('%i batches finished!' %step)
      offset = step * FLAGS.test_batch_size
      test_image_batch = test_image_array[offset:offset+FLAGS.test_batch_size, ...]
      batch_prediction_array = sess.run(predictions, feed_dict={self.test_image_placeholder: test_image_batch})
      prediction_array = np.concatenate((prediction_array, batch_prediction_array))

    # If test_batch_size is not a divisor of num_test_images
    if remain_images != 0:
      self.test_image_placeholder = tf.placeholder(dtype=tf.float32, shape=[remain_images, img_height, img_width, img_depth])
      logits = resnet(self.test_image_placeholder, FLAGS.num_residual_blocks, reuse=True)
      predictions = tf.nn.softmax(logits)
      test_image_batch = test_image_array[-remain_images:, ...]
      batch_prediction_array = sess.run(predictions, feed_dict={self.test_image_placeholder: test_image_batch})
      prediction_array = np.concatenate((prediction_array, batch_prediction_array))

    return prediction_array

# Helper functions 

  def loss(self, logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels)
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    return cross_entropy_mean

  def top_k_error(self, predictions, labels, k):
    batch_size = predictions.get_shape().as_list()[0]
    top1_precision = tf.nn.in_top_k(predictions, labels, k=1)
    num_correct = tf.reduce_sum(tf.cast(top1_precision, tf.float32))
    # self.train_accuracy = (num_correct) / float(batch_size)
    return ((batch_size - num_correct) / float(batch_size)) , (num_correct) / float(batch_size)

  def train_operation(self, global_step, total_loss, top1_error,accuracy):
    tf.summary.scalar('learning_rate', self.lr_ph)
    tf.summary.scalar('train_loss', total_loss)
    tf.summary.scalar('train_top1_error', top1_error)
    tf.summary.scalar('train_accuracy',accuracy)

    # To help compute moving avg of training loss and training error 
    ema = tf.train.ExponentialMovingAverage(FLAGS.train_ema_decay, global_step)
    train_ema_op = ema.apply([total_loss, top1_error])
    tf.summary.scalar('train_top1_error_avg', ema.average(top1_error))
    tf.summary.scalar('train_loss_avg', ema.average(total_loss))

    opt = tf.train.MomentumOptimizer(learning_rate=self.lr_ph, momentum=0.9)
    train_op = opt.minimize(total_loss, global_step=global_step)
    return train_op, train_ema_op

  def validation_operation(self, validation_step, top1_error, loss,accuracy):
    ema = tf.train.ExponentialMovingAverage(0.0, validation_step) # Decay = 0.0 --> returns original error
    ema2 = tf.train.ExponentialMovingAverage(0.95, validation_step)

    val_op = tf.group(validation_step.assign_add(1), ema.apply([top1_error, loss]), ema2.apply([top1_error, loss]))
    top1_error_val = ema.average(top1_error)
    top1_error_avg = ema2.average(top1_error)
    loss_val = ema.average(loss)
    loss_val_avg = ema2.average(loss)

    tf.summary.scalar('val_top1_error', top1_error_val)
    tf.summary.scalar('val_top1_error_avg', top1_error_avg)
    tf.summary.scalar('val_loss', loss_val)
    tf.summary.scalar('val_loss_avg', loss_val_avg)
    tf.summary.scalar('val_accuracy',accuracy)
    return val_op

  def full_validation(self, loss, top1_error, session, vali_data, vali_labels, batch_data,batch_label):
    num_batches = 10000 // FLAGS.validation_batch_size
    order = np.random.choice(10000, num_batches * FLAGS.validation_batch_size)
    vali_data_subset = vali_data[order, ...]
    vali_labels_subset = vali_labels[order]

    loss_list = []
    error_list = []

    for step in range(num_batches):
      offset = step * FLAGS.validation_batch_size
      feed_dict = {self.image_ph: batch_data, self.label_ph: batch_label,
        self.valid_image_ph: vali_data_subset[offset:offset+FLAGS.validation_batch_size, ...],
        self.valid_label_ph: vali_labels_subset[offset:offset+FLAGS.validation_batch_size],
        self.lr_ph: FLAGS.init_lr}
      loss_value, top1_error_value = session.run([loss, top1_error], feed_dict=feed_dict)
      loss_list.append(loss_value)
      error_list.append(top1_error_value)
      
    return np.mean(loss_list), np.mean(error_list)

In [0]:
train = Train_ResNet()
train.train()

Shuffling
Shuffling
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Training
-----------------------------------
2020-05-05 15:09:57.888828: step 0, loss = 2.3296 (36.0 examples/sec; 3.560 sec/batch)
Train top1 err =  0.9140625
Valid top1 err = 0.9240
Valid loss =  2.3055503
-----------------------------------
2020-05-05 15:18:11.461123: step 400, loss = 2.0042 (106.8 examples/sec; 1.199 sec/batch)
Train top1 err =  0.515625
Valid top1 err = 0.5120
Valid loss =  1.9726697
-----------------------------------
2020-05-05 15:26:24.081365: step 800, loss = 1.9956 (103.0 examples/sec; 1.243 sec/batch)
Train top1 err =  0.4921875
Valid top1 err = 0.4120
Valid loss =  1.8941118
-----------------------------------
2020-05-05 15:3

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
%tensorboard --logdir logs_test_3